# CS598 Deep Learning for Healthcare Final Project
## Reproduction of Deepr: A Convolutional Net for Medical Records
### Juan Alvarez Martinez, Shane Sepac

## Reproducibility Summary
We implemented *Deepr: A Convolutional Net for Medical Records* and achieved an AUC in the model validation consistent and if not better than the original paper reported. The MIMIC-3 dataset was used for testing against the Deepr paper. The python library *Pyhealth* was used to ingest the MIMIC-3 dataset in order to leverage patient, visit and event level data. These data were mapped into "sentences," as referenced in the paper, which were formed by capturing the ICD-9 diagnoses and procedure codes for events within a given visit into "phrases", and each phrase concatenated between a timestamp indicating duration between patient visits. After generating sentences according to every patient's EMR records, we generated word embeddings for each sentence via Word2Vec (skip-gram), which was leveraged to help capture the context of surrounding words. The embeddings were pushed through our Deepr model implementation, which consists of convolutional, relu, max pooling and linear classification layers, which produced a binary classification of unplanned readmission likelihood. The data used to train the model was a 1-1 mixture of unplanned readmitted and non-readmitted patients. During validation, we achieved results consistent with the Deepr paper, with an AUC of ~0.85.

## Load MIMIC-III Dataset. 
Several csv files are needed from the MIMIC-III dataset: ADMISSIONS, PATIENTS, DIAGNOSES_ICD, and PROCEDURES_ICD. These files can be loaded automatically out of S3, or you can place them in `<project_root>/mimic3`. 
- If loading out of S3, ensure you have all environment variables from .env.sample copied and instantiated in a .env file!

In [1]:
# install the required dependencies
%pip install boto3 python-dotenv pandas pyhealth matplotlib

Note: you may need to restart the kernel to use updated packages.


### Get MIMIC-3 Data
Attempt to load MIMIC-3 data out of S3 if the relevant CSV files are not already in the mimic3 folder at the project root.

In [2]:
import os
from utils import copy_file_from_s3

data_folder = "mimic3"
required_files = ["ADMISSIONS.csv", "PATIENTS.csv", "DIAGNOSES_ICD.csv", "PROCEDURES_ICD.csv", "TRANSFERS.csv"]

for i, fn in enumerate(required_files):
  if not os.path.exists(f"{data_folder}/{fn}"):
    print(f"Cannot find {fn} in {data_folder}, trying to download from S3...")
    copy_file_from_s3(fn, data_folder)
  else:
    print(f"Found {fn}...")

Found ADMISSIONS.csv...
Found PATIENTS.csv...
Found DIAGNOSES_ICD.csv...
Found PROCEDURES_ICD.csv...
Found TRANSFERS.csv...


In [3]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_ds = MIMIC3Dataset("./mimic3/", ["DIAGNOSES_ICD", "PROCEDURES_ICD"]) #pyhealth does not support mapping ICD-9 to ICD-10 codes.

mimic3_ds.info()
mimic3_ds.stat()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PROCEDURES_ICD: 4.0711



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

## Sequencing EMR: Creating Sentences representing patient episodes
Per Deepr, an EMR must be translated into a sentence for use downstream the model. An EMR is a sequence of time-stamped visit episodes. Each episode involves a series of diagnoses and treatments, called a phrase. Each phrase is separated by a time interval equal to `(0–1], (1–3], (3–6], (6–12], and 12+` or `TRANSFER`, with the latter indicating a transfer between care providers (separate departments within the same hospital or between hospitals.) Infrequent words are coded with `RAREWORD`, which indicates the word has appeared <100 times. Per the Deepr paper, an example sentence looks as follows:

```
1910 Z83 911 1008 D12 K31 1-3m R94 RAREWORD H53 Y83 M62 Y92 E87 T81 RAREWORD RAREWORD 1893 D12 S14 738 1910 1916 Z83 0-1m T91 RAREWORD Y83 Y92 K91 M10 E86 6-12m K31 1008 1910 Z13 Z83.
```

Note: In the sentence above, diagnoses are in ICD-10 format (a character followed by digits) and procedures are in digits. 

The MIMIC-3 dataset provides ICD-9 codes, and these will be used, but the level-3 variant of them for consistency with the original paper. It can also be noted that the encounter and discharge datetimes for visits are between the years 2100-2200 in order to deidentify patients, however, the time interval between visits is indeed preserved.


In [4]:
'''
Find rare words (diagnoses and procedures with counts of less than 100)
'''
word_cnts = {}

for i, p in enumerate(mimic3_ds.patients.values()):
  words = []

  for _, v in p.visits.items():
    for e in v.get_event_list('DIAGNOSES_ICD'):
      words.append(e.code)

    for e in v.get_event_list('PROCEDURES_ICD'):
      words.append(e.code)

  for word in words:
    # If the word is already in the dictionary, increment the count
    if word in word_cnts:
        word_cnts[word] += 1
    # Otherwise, add the word to the dictionary with a count of 1
    else:
        word_cnts[word] = 1

In [5]:
'''
Append to pyhealth's record of visits so that ADMISSION_TYPE and ADMISSION_LOCATION data are available. The former is used later in the model training
to target against readmission (by looking at non-elective admits) and the latter is used to discern intra/inter hospital transfers, which is needed to build 
the sentences already described (i.e. to help build sentences using the TRANSFER keyword.)
'''
import pandas as pd

admissions_df = pd.read_csv("./mimic3/ADMISSIONS.csv")
admissions_df = admissions_df.set_index("HADM_ID")

for i, p in enumerate(mimic3_ds.patients.values()):
  # Sort patient visits by encounter_time
  for i, v in enumerate(p.visits.items(),):
    res = admissions_df.loc[int(v[1].visit_id)]

    v[1].attr_dict["ADMISSION_TYPE"] = res["ADMISSION_TYPE"]
    v[1].attr_dict["ADMISSION_LOCATION"] = res["ADMISSION_LOCATION"]

In [6]:
from utils import timedelta_to_interval
import random
import json

'''
Translate EMRs into sentences outlined by the paper. A sentence consists of phrases, which are randomly shuffled diagnosis and procedure codes, separated by the 
time interval between visits, if the time interval exists. Sentences should have 100 words max.

While looping over each patient:
  1. Sort visits by encounter_time
  2. Find the time interval between each visit and generate its relevant string word
  3. Build arrays of diagnosis and procedure codes for each visit, replacing ICD-10 codes with less than 100 usages with RAREWORD
  4. Randomly shuffle each array of diagnosis and procedure codes, then append the time interval string if available. This represents a phrase.
    Concat each phrase to an array, which will be concatenated to form the final sentence. If the concatenation would form a sentence longer
    than 100 words, min(100, words(sentence)) is adhered to.
'''
sentences = []
for i, p in enumerate(mimic3_ds.patients.values()):

  # Sort patient visits by encounter_time
  sorted_visits = sorted(p.visits.items(), key=lambda v: v[1].encounter_time) # sort by encounter time in order to guage interval between visits

  # Generate timestamps in between visits
  discharge_times = [visit[1].discharge_time for visit in sorted_visits[:-1]]
  encounter_times = [visit[1].encounter_time for visit in sorted_visits[1:]]

  time_intervals = [
      t2 - t1
      for t1, t2 in zip(discharge_times, encounter_times)
  ]

  # Convert timestamps to month intervals as specified in paper
  time_interval_strs = timedelta_to_interval(time_intervals)

  # event_diagnoses_ls = (visit, diagnoses_codes)
  event_diagnoses_ls = []

  # event_procedures_ls = (visit, procedure_codes)
  event_procedures_ls = []

  # Helper function to create arrays with RAREWORD using list comprehension
  def handle_event(event_list, word_cnts):
      return ["RAREWORD" if e.code in word_cnts and word_cnts[e.code] < 100 else e.code for e in event_list]

  # build arrays of diagnoses and procedures on a visit level, add to event_diagnoses_ls or event_procedures_ls
  for _, v in sorted_visits:
      visit_diagnoses = handle_event(v.get_event_list('DIAGNOSES_ICD'), word_cnts)
      event_diagnoses_ls.append(visit_diagnoses)

      visit_procedures = handle_event(v.get_event_list('PROCEDURES_ICD'), word_cnts)
      event_procedures_ls.append(visit_procedures)


  # Randomly shuffle diagnosis and procedure codes and append a time interval after, if available. Ensure the output sentence will not be more than 100 words.
  arrs = []
  word_cnt = 0
  for i, vd in enumerate(event_diagnoses_ls):
      arr = vd + event_procedures_ls[i]
      random.shuffle(arr)
      if i < len(time_interval_strs):
          arr.append(time_interval_strs[i])

      new_word_cnt = word_cnt + len(arr)

      if new_word_cnt > 100:
          # Calculate the number of elements needed to reach exactly 100 words
          elements_needed = 100 - word_cnt

          # Take a subset of arr to make new_word_cnt equal 100
          arr = arr[:elements_needed]
          arrs.append(arr)
          break

      arrs.append(arr)
      word_cnt = new_word_cnt

  # Combine all codes and time interval to create a phrase, representing a visit
  phrases = [" ".join(arr) for arr in arrs]

  # Combine all phrases to create a sentence, representing a sequence as outlined by the paper
  sentence = " ".join(phrases)
  sentences.append(sentence)

# output to json file
output_dir = "data"
output_filename = "sentences.json"

os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, output_filename), "w") as json_file:
  json.dump(sentences, json_file)

### Test that output sentences satisfy the following conditions:
- There is a sentence for each patient
- Each sentence is capped to max 100 words
- Multi visit patients have visits separated by a timestamp
- Words should not exist in their ICD-10 form if used less than 100 times (should be replaced with RAREWORD)


In [7]:
import re

# There should be one sentence per patient
num_patients = len(mimic3_ds.patients)
num_sentences = len(sentences)
assert(num_patients == num_sentences)

# There should be max 100 words per sentence
word_lengths = map(lambda s: len(s.split()), sentences)
assert(max(list(word_lengths)) <= 100)

# There should be no word in any of the sentences that is present less than 100 times
rarewords = [word for word, count in word_cnts.items() if count < 100]
for sentence in sentences:
  words_of_sentence = sentence.split()
  rareword_violations = list(filter(lambda w: w in word_cnts and word_cnts[w] < 100, words_of_sentence))
  assert(len(rareword_violations) == 0)


# Patients with multiple visits should have timestamps separating their visits i.e. 1-3m or 12+m #TODO: Add TRANSFER to regex
pattern = re.compile(r"[-+]")
for i, p in enumerate(mimic3_ds.patients.values()):
    if len(p.visits) > 1:
        if not pattern.search(sentences[i]):
            print(f"Failed assertion for sentences[{i}]: '{sentences[i]}'")
            assert(False)


## Training Word2Vec

Word2Vec is a machine learning algorithm used to generate word embeddings that capture the semantic meaning of words. In the context of medical data analysis, Word2Vec can be used to analyze the relationship between diagnoses and treatments given an underlying disease. Instead of text, however, we can use ICD9 codes as words to create the embeddings. By training the model on a large corpus of medical records, we can identify relationships between different diagnoses and treatments, and group them into clusters denoting comorbidities. These clusters can help identify patterns in the data that may be useful for predicting disease outcomes and developing targeted treatment plans.

In [8]:
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


class Word2VecDataset(Dataset):
    def __init__(self, dataset, tok2id, wsize=3):
        self.wsize = wsize
        self.tok2id = tok2id

        self.dataset = [
            ctx
            for sentence in dataset for ctx in self.get_contexts(sentence)
        ]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

    def get_contexts(self, sentence):
        moving_window = []

        for word_ix, word in enumerate(sentence):
            target = self.tok2id[word]
            window = [
                word_ix + win_ix
                for win_ix in range(-self.wsize, self.wsize + 1)
                if (
                    word_ix + win_ix >= 0
                    and word_ix + win_ix < len(sentence)
                    and win_ix != 0
                )
            ]

            moving_window += [
                (target, self.tok2id[sentence[win_word]]) for win_word in window
            ]

        return moving_window


class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.prediction = nn.Linear(embedding_size, vocab_size, bias=False)

    def forward(self, input):
        hidden = self.embedding(input)
        logits = self.prediction(hidden)

        return logits


dataset = [sentence.split(" ") for sentence in sentences]
vocab = set([word for sentence in dataset for word in sentence])

id2tok = dict(enumerate(vocab))
tok2id = {token: id for id, token in id2tok.items()}

word2vec_dataloader = DataLoader(
    Word2VecDataset(dataset, tok2id),
    batch_size=64,
    shuffle=True
)

word2vec_model = Word2Vec(len(vocab), embedding_size=100)

In [ ]:
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt


LR = 3e-4
EPOCHS = 20

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(word2vec_model.parameters(), lr=LR)
progress_bar = tqdm(range(EPOCHS * len(word2vec_dataloader)))

running_loss = []

for epoch in range(EPOCHS):
    epoch_loss = 0
    for center, context in word2vec_dataloader:
        optimizer.zero_grad()

        logits = word2vec_model(input=context)
        loss = loss_fn(logits, center)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()
        progress_bar.update(1)

    epoch_loss /= len(word2vec_dataloader)
    running_loss.append(epoch_loss)

plt.plot(running_loss)

In [ ]:
weights = word2vec_model.state_dict()
torch.save(weights, os.path.join(output_dir, "word2vec.pt"))

## Convolutional Network
Now that Word2Vec has created embeddings for the sentences we generated in the first step, we are ready to pass the embeddings through a convolutional layer, followed by max pooling.

First, we will generate the train and val loaders. This will first require us to create the validation set. In Deepr, we try to predict the likelihood of unplanned re-admission after both 3 and 6 months. Unplanned re-admission is coded in the database as an unplanned or emergency with the status not equal to "elective" or a transfer from another hospital.

Next, we will define a model containing a convolutional network, ReLU and max pooling layer.

Deepr reported the following parameters were determined to be optimal for the model:

`m = 100, d = 1, motif size = 3, 4, and 5, n = 100 number of epochs = 10, minibatch size = 64, and l2 regularization λ = 1.0.`

In [9]:
'''
Create validation dataset, train and val loaders for training Deepr.
'''

val_readmission_time_days = 180 # The threshold for flagging unplanned readmission for validation set
risk_group = []
remaining_group = [] #patients not put into the risk group
empty_sentence_cnt = 0
for i, p in enumerate(mimic3_ds.patients.values()):

  # Sort patient visits by encounter_time
  sorted_visits = sorted(p.visits.items(), key=lambda v: v[1].encounter_time) # sort by encounter time in order to guage interval between visits

  # Generate timestamps in between visits
  discharge_times = [visit[1].discharge_time for visit in sorted_visits[:-1]]
  encounter_times = [visit[1].encounter_time for visit in sorted_visits[1:]]

  time_intervals = [
      t2 - t1
      for t1, t2 in zip(discharge_times, encounter_times)
  ]

  visit_types = [visit[1].attr_dict["ADMISSION_TYPE"] for visit in sorted_visits[1:]]

  unplanned_readmissions = [
      i_vt
      for i_vt, (interval, visit_type) in enumerate(zip(time_intervals, visit_types))
      if interval.days <= val_readmission_time_days and visit_type == "EMERGENCY"
  ]

  if sentences[i]:
    if unplanned_readmissions:
      risk_group.append(sentences[i])
    else: 
      remaining_group.append(sentences[i])
  else:
    empty_sentence_cnt += 1

In [10]:
# Risk group patients should be separate from the candidate group, and sum to the length of the total dataset
assert(len(mimic3_ds.patients.values()) == len(risk_group) + len(remaining_group) + empty_sentence_cnt)

In [11]:
import torch
from torch.utils.data import Dataset

'''
Dataset for Deepr model. __getitem__ returns the word indices for a given sentence. 
DeeprModel will then look up the word embeddings by the index in the embedding table.
'''

class DeeprDataset(Dataset):
    def __init__(self, x_data, y_data, word2index):
        self.x_data = x_data
        self.y_data = y_data
        self.word2index = word2index

    def get_indices(self, sentence):
        return torch.tensor([self.word2index[word] for word in sentence.split()])

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.get_indices(self.x_data[idx])
        y = self.y_data[idx]
        return x, y

In [12]:
from torch.nn.utils.rnn import pad_sequence

'''
The sentences in each batch will likely not all be the same size, and thus we need to pad the sentences. 
DeeprDataset returns the indicies of the words in each sentence. We will padd these vectors of indicies with 0s.
It is important later on ensure that index 0 of the embedding table is a torch.zero(embedding_dim), as these are just placeholder values,
and to mask them out later on.
'''

def pad_collate(batch):
    x_data, y_data = zip(*batch)
    x_data = list(x_data)

    # TODO: Receive max_len as an argument
    padded_fst = torch.zeros(100)  # max_len
    padded_fst[: x_data[0].shape[0]] = x_data[0]
    x_data[0] = padded_fst

    x_data_padded = pad_sequence(x_data, batch_first=True, padding_value=0)

    # Create binary mask
    mask = (x_data_padded != 0).float()

    # Convert from tuple to tensor
    y_data_tensor = torch.tensor(y_data, dtype=torch.long)

    return x_data_padded.long(), y_data_tensor, mask

In [13]:
import random
from torch.utils.data import random_split, DataLoader


'''
Generate the training and validation sets. The training data is a 1-1 mixture of risk and control patients, where x 
is the sentence of the patient, and y is the corresponding binary label of readmission (1=readmit, 0=non-readmit)

We also ensure that index 0 of the embedding table is associated with the word "<pad>", so that the padding index has an associated word, 
as it was not learned in Word2Vec, but will be used in the Deepr model later on.
'''

train_ratio = 0.8

control_group = random.sample(remaining_group, len(risk_group))
control_labels = [0] * len(control_group)
control_zip = list(zip(control_group, control_labels))

risk_labels = [1] * len(risk_group)
risk_zip = list(zip(risk_group, risk_labels))

all_zip = list(control_zip + risk_zip)

x, y = zip(*all_zip)

# Shift all elements up, and add the padding word at index 0
word2index = {word: idx + 1 for idx, word in enumerate(tok2id)}
word2index["<pad>"] = 0

train_dataset, cv_dataset, val_dataset = random_split(DeeprDataset(x, y, word2index), [0.64, 0.16, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=pad_collate)
cv_dataloader = DataLoader(cv_dataset, batch_size=64, shuffle=False, collate_fn=pad_collate)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=pad_collate)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

'''
DeeprModel does the following things:

1. During initialization, updates the embedding table learned from Word2Vec to include the embedding for padding.
2. Pretrains an embedding layer with the weights from Word2Vec.
3. Initializes the model with hyperparameters from the Deepr paper, mentioned at the beginning of this section.
4. Passes the embeddings through convolutional, max pooling and a linear classifier.
'''

class DeeprModel(nn.Module):
    def __init__(self, num_filters, kernel_size, num_classes, max_len, vocab_size, embedding=None, with_cnn=True, with_attn=True):
        super(DeeprModel, self).__init__()

        self.max_len = max_len
        self.with_cnn = with_cnn
        self.with_attn = with_attn
        self.vocab_size = vocab_size
        self.num_filters = num_filters

        # Layers
        if embedding:
            self.embedding = embedding
            in_channels = self.embedding.weight.shape[1]
        else:
            self.embedding = None
            in_channels = vocab_size

        if with_cnn:
            self.conv = nn.Conv1d(in_channels, num_filters, kernel_size, padding='same')
            self.pool = nn.AdaptiveMaxPool1d(1)
            out_channels = num_filters
        else:
            self.rnn = nn.GRU(in_channels, num_filters, batch_first=True, bidirectional=True)
            self.attn = nn.Linear(2 * num_filters * max_len, max_len)
            out_channels = 2 * num_filters

        self.fc = nn.Linear(out_channels, num_classes)

    def forward(self, x, mask):
        if self.embedding:
            x = self.embedding(x)  # Shape: (batch_size, sequence_length, embedding_dim)
        else:
            x = nn.functional.one_hot(x, num_classes=self.vocab_size).float()  # Shape: (batch_size, sequence_length, vocab_size)

        if self.with_cnn:
            x = x.transpose(1, 2)  # conv requires input_channels to be dim 1. Shape: (batch_size, embedding_dim, sequence_length)
            x = F.relu(self.conv(x))  # Shape: (batch_size, num_filters, sequence_length)
            x = x * mask.unsqueeze(1)  # Apply mask
            relu_output = x.detach().cpu().numpy() # used in motif detection analysis later; not part of training/validation
            x = self.pool(x).squeeze(2)  # Shape: (batch_size, num_filters)
        else:
            sequence_lengths = mask.sum(dim=1).long()
            x_packed = nn.utils.rnn.pack_padded_sequence(x, sequence_lengths, batch_first=True, enforce_sorted=False)
            output, h_n = self.rnn(x_packed)

            if self.with_attn:
                # Force the output of the RNN to have shape: (batch_size, max_len, 2 * num_filters)
                output = nn.utils.rnn.unpack_sequence(output)
                padded_sen = torch.zeros(self.max_len, 2 * self.num_filters)
                padded_sen[: output[0].shape[0]] = output[0]
                output[0] = padded_sen
                padded_output = pad_sequence(output, batch_first=True)

                # Calculate masked attention scores
                attn_weights = self.attn(padded_output.view(padded_output.shape[0], padded_output.shape[1] * padded_output.shape[2]))
                attn_weights[mask == 0] = -1e9
                attn_score = F.softmax(attn_weights, dim=1)[:, :, None]

                # Apply attention
                x = (attn_score * padded_output).sum(dim=1)
            else:
                x = torch.cat((h_n[-1], h_n[-2]), dim=1)  # Shape: (batch_size, 2 * num_filters)

            relu_output = None

        x = self.fc(x)  # Shape: (batch_size, num_classes)
        return x, relu_output

## Training and validating DeeprModel

In [15]:
'''
This function is a helper function to take the output of the filter responses (ReLU(CNN(x)) in Deepr's forward method) and map them back to the original sentences. 
Strong filter responses are indicative that the motif within the convolutional window contributes towards the model predicting unplanned hospital readmission.
'''

# Remaps tok2id to include the padding index used in Deepr.
index2WordValidation = {idx+1: word for idx, word in enumerate(tok2id)}
index2WordValidation[0] = '<pad>'

# Helper function
def normalize_array(arr, max_val):
    min_val = 0
    normalized = [1 + (x - min_val) * (2 - 1) / (max_val - min_val) for x in arr]
    return normalized

def getMotifs(batch_x, relu_output, probabilities, filter_num=0):
    '''
    Gets the top 10 most likely readmit and 10 most unlikely readmits for the given batch. Used to analyze the motifs most strongly contributing to readmission.

    batch_x contains the batch in the validation loop (shape: batch_size, padded_sentence_length)
    relu_output contains the filter responses from Deepr (shape: batch_size, num_filters, padded_sentence_length)
    probabilities contains the probabilities of the sentences being in the positive class (positive readmit probability). (shape: batch_size)
    filter_num is a 0 indexed number corresponding to which filter from the CNN the motifs should be pulled from.
    '''

    output_list = []
    for i in range(batch_x.size(0)):
      # Remap indices from tensors in batch back to their original sentences
      word_list = [index2WordValidation[idx.item()] for idx in batch_x[i]]
      relu_values = relu_output[i][filter_num].tolist()
      is_positive = probabilities[i] > 0.5

      instance_data = {
          "sentence": " ".join(word_list),
          "is_positive": bool(is_positive),
          "is_positive_score": probabilities[i].item(),
          "raw_motif_values": relu_values
      }

      output_list.append(instance_data)

    return output_list

def normalize_motifs(motifs, max_motif_val):
  for i in range(len(motifs)):
    normalized_motifs = normalize_array(motifs[i]["raw_motif_values"], max_motif_val)
    motifs[i]["normalized_motif_values"] = normalized_motifs
  return motifs


In [29]:
from collections import deque
import itertools
import json
import numpy as np
import os
from sklearn.metrics import roc_auc_score
import torch


'''
We first initialize the model's hyperparameters. Then, we load the saved weights from Word2Vec into a new word2vec_model instance. 
We initialize a DeeprModel instance, CrossEntropyLoss function, and SGD optimizer, the later two being specified in the Deepr paper.
In the training loop, we compare the logits computed in the forward method of the model against the labels we computed in other steps,
compute the loss, back propogate and update weights.

In validating the data, we compute validation loss and AUC.

'''


def train_model(model, criterion, optimizer, train_data, cv_data):
    model.train()

    for batch_x, batch_y, mask in train_data:
        optimizer.zero_grad()
        logits, _ = model(batch_x, mask)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()

    val_loss = 0
    all_logits = []
    all_labels = []

    with torch.no_grad():
        model.eval()

        for batch_x, batch_y, mask in cv_data:
            logits, _ = model(batch_x, mask)

            loss = criterion(logits, batch_y)
            val_loss += loss.item()

            # Get probabilities for the positive class
            probabilities = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()

            all_logits.extend(probabilities)
            all_labels.extend(batch_y.cpu().numpy())

    val_loss /= len(cv_data)
    auc = roc_auc_score(all_labels, all_logits)

    return {'state_dict': model.state_dict(), 'loss': val_loss, 'auc': auc}


def get_min_std(models):
    aucs = torch.tensor([model['auc'] for model in models])

    return torch.min(aucs), torch.std(aucs)


# Hyperparameters
learning_rate = 1e-3
l2_reg = 1.0

# Initialize the model
seq_max_len = 100
num_filters = 64
filter_size = 3
num_classes = 2  # For binary classification
embedding_dim = 100
with_cnn = True
with_attn = True


# Load weights from .pt file
vocab_size = len(vocab)
word2vec_model = Word2Vec(vocab_size, embedding_dim)
weights_path = os.path.join(output_dir, "word2vec_time.pt")
saved_weights = torch.load(weights_path)

word2vec_model.load_state_dict(saved_weights)

word_embeddings_dict = {
    word: word2vec_model.embedding(torch.tensor(tok2id[word])).detach().numpy()
    for word in tok2id
}

# Prepare the embedding matrix; this takes the ouput of word2vec but inserts "<pad>" string, assigning an empty embedding to it
embedding_matrix = torch.zeros(len(word_embeddings_dict) + 1, embedding_dim)
for word, index in word2index.items():
    if word != "<pad>":
        # embedding_matrix[index] = torch.rand(1, embedding_dim)
        embedding_matrix[index] = torch.tensor(word_embeddings_dict[word])

deepr_embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False, padding_idx=0)
model = DeeprModel(
    num_filters,
    filter_size,
    num_classes,
    seq_max_len,
    len(word_embeddings_dict) + 1,
    embedding=deepr_embedding,
    with_cnn=with_cnn,
    with_attn=with_attn
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=l2_reg)

potential_models = deque()

for epoch in range(10):
    new_model = train_model(model, criterion, optimizer, train_dataloader, cv_dataloader)

    potential_models.append(new_model)

    print(f"Epoch {epoch + 1}, Loss: {new_model['loss']:.4f}, AUC: {new_model['auc']:.4f}")

min, std = get_min_std(potential_models)
threshold = min - std

for epoch in itertools.count(start=11):
    potential_models.popleft()

    new_model = train_model(model, criterion, optimizer, train_dataloader, cv_dataloader)

    if new_model['auc'] < threshold:
        break

    potential_models.append(new_model)

    min, std = get_min_std(potential_models)

    if std < 1e-4:
        break

    threshold = min - std

    print(f"Epoch {epoch}, Loss: {new_model['loss']:.4f}, AUC: {new_model['auc']:.4f}")

min_ix = int(torch.tensor([m['auc'] for m in potential_models]).argmin().item())
model.load_state_dict(potential_models[min_ix]['state_dict'])

max_motif_val = 0.0
motifs_data = []

val_loss = 0
all_logits = []
all_labels = []

# Validation
with torch.no_grad():
    model.eval()

    for batch_x, batch_y, mask in val_dataloader:
        logits, relu_output = model(batch_x, mask)
        
        loss = criterion(logits, batch_y)
        val_loss += loss.item()

        # Get probabilities for the positive class
        probabilities = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()

        if with_cnn:
            # Output motifs data for each sentence in the batch
            batch_motifs = getMotifs(batch_x, relu_output, probabilities)
            motifs_data.extend(batch_motifs)

            # find max value of the batch and update the max motif value across the validation set
            max_batch_motif_val = np.max(relu_output)
            max_motif_val = max(max_batch_motif_val, max_motif_val)

        all_logits.extend(probabilities)
        all_labels.extend(batch_y.cpu().numpy())

val_loss /= len(val_dataloader)
auc = roc_auc_score(all_labels, all_logits)

print(f"Test Loss: {val_loss:.4f}, Test AUC: {auc:.4f}")

if with_cnn:
  # Normalize the motifs across the validation set
  motifs_data_normalized = normalize_motifs(motifs_data, max_motif_val)
  # sort the motifs from most at risk of unplanned readmit to least
  motifs_data_normalized = sorted(motifs_data_normalized, key=lambda x: x['is_positive_score'], reverse=True)

  # Save the sentence level motifs to a JSON file
  with open(os.path.join(output_dir, "motifs_data.json"), "w") as outfile:
      json.dump(motifs_data_normalized, outfile)

Epoch 1, Loss: 0.6993, AUC: 0.3886
Epoch 2, Loss: 0.6951, AUC: 0.4521
Epoch 3, Loss: 0.6932, AUC: 0.5165
Epoch 4, Loss: 0.6921, AUC: 0.5755
Epoch 5, Loss: 0.6916, AUC: 0.6268
Epoch 6, Loss: 0.6914, AUC: 0.6674
Epoch 7, Loss: 0.6914, AUC: 0.7003
Epoch 8, Loss: 0.6914, AUC: 0.7267
Epoch 9, Loss: 0.6916, AUC: 0.7475
Epoch 10, Loss: 0.6917, AUC: 0.7643
Epoch 11, Loss: 0.6919, AUC: 0.7781
Epoch 12, Loss: 0.6921, AUC: 0.7895
Epoch 13, Loss: 0.6922, AUC: 0.7987
Epoch 14, Loss: 0.6924, AUC: 0.8062
Epoch 15, Loss: 0.6925, AUC: 0.8126
Epoch 16, Loss: 0.6926, AUC: 0.8178
Epoch 17, Loss: 0.6927, AUC: 0.8227
Epoch 18, Loss: 0.6928, AUC: 0.8268
Epoch 19, Loss: 0.6929, AUC: 0.8302
Epoch 20, Loss: 0.6929, AUC: 0.8330
Epoch 21, Loss: 0.6930, AUC: 0.8353
Epoch 22, Loss: 0.6930, AUC: 0.8372
Epoch 23, Loss: 0.6930, AUC: 0.8393
Epoch 24, Loss: 0.6930, AUC: 0.8410
Epoch 25, Loss: 0.6931, AUC: 0.8423
Epoch 26, Loss: 0.6931, AUC: 0.8429
Epoch 27, Loss: 0.6931, AUC: 0.8431
Epoch 28, Loss: 0.6931, AUC: 0.8432
E